<a href="https://colab.research.google.com/github/NarmadaBalasooriya/Video-background-foreground-composing/blob/main/Video%20Background%20and%20foreground%20composing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Change the working directory

Change this according to the VM

In [ ]:
%cd "Path to your Google Driver Location"

### Import the rquired libraries

In [ ]:
!pip install scikit-video

     |████████████████████████████████| 2.3 MB 8.5 MB/s 


**Required**

In [ ]:
#!/usr/bin/env python3
import cv2
import numpy as np
from moviepy.editor import *
import matplotlib.pyplot as plt
import skvideo.io  
import skimage.exposure
from skimage import morphology
from PIL import Image, ImageFilter, ImageOps, ImageChops

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2924544/45929032 bytes (6.4%)6496256/45929032 bytes (14.1%)10018816/45929032 bytes (21.8%)13492224/45929032 bytes (29.4%)16801792/45929032 bytes (36.6%)20299776/45929032 bytes (44.2%)23748608/45929032 bytes (51.7%)27230208/45929032 bytes (59.3%)30736384/45929032 bytes (66.9%)34275328/45929032 bytes (74.6%)37576704/45929032 bytes (81.8%)41132032/45929032 bytes (89.6%)

### Functions to compose the videos frame by frame

#### Function 1 - compose_video

In [ ]:
def compose_video(fore_fr, mask_fr, back_fr, pos_x, pos_y, disp):
    
  mask_h = mask_fr.shape[0]
  mask_w = mask_fr.shape[1]

  back_final = back_fr.copy()

  # Check for the sizes to avoid errors
  if(pos_x + mask_w)>back_fr.shape[1] and (pos_y + mask_h)>back_fr.shape[0]:
      raise ValueError("Error in positioning. Try repositioning")
  
  # Get the ROI of the background for the mask positioning
  bg_for_mask = back_fr[pos_y:pos_y+mask_h, pos_x:pos_x+mask_w, :]
  
  # black and white pixel interchange
  mask_inv = cv2.bitwise_not(mask_fr)
  # Thresholding the pixel values to binary where 255 is equivalent to 1 in
  # uint8 datatype
  #mask_thresh = cv2.threshold(mask_inv, 127, 255, cv2.THRESH_BINARY)[1]
  mask_thresh = mask_inv

  # Morphological close and dilate to fill the gaps
  kernel = np.ones((3,3), np.uint8)
  mask_C = cv2.morphologyEx(mask_thresh, cv2.MORPH_CLOSE, kernel)
  mask_D = cv2.dilate(mask_C, kernel)

  # Convert white pixels to transparent
  mask_frn = cv2.cvtColor(mask_D, cv2.COLOR_RGB2RGBA)

  white = np.sum(mask_frn[:,:,:3], axis=2)
  white_mask = np.where(white == 255*3, 1, 0)
  alpha = np.where(white_mask, 0, mask_frn[:,:,-1])

  mask_frn[:,:,-1] = alpha 

  # Convert the mask to PIL image to be used for PIL composite function
  mask_pil = Image.fromarray(mask_frn)
  
  # Convert the background and foreground frames to PIL Image
  back_pil = Image.fromarray(bg_for_mask)
  fore_pil = Image.fromarray(fore_fr)

  # Apply PIL Image composite to foreground and background with the mask
  dst_pil = Image.composite(fore_pil, back_pil, mask_pil)
  
  # Convert the composed frame to numpy
  dst_np = np.asarray(dst_pil)

  # Restore the composed ROI into the original frame
  back_final[pos_y:pos_y+mask_h, pos_x:pos_x+mask_w, :] = dst_np

  # Display the intermediate results
  if(disp):
    f, axarr = plt.subplots(6,2, figsize=(40, 100))
    #plt.axis('off')

    # foreground frame
    axarr[0,0].set_title('Foreground frame', fontsize=20)
    axarr[0,0].imshow(fore_pil)
    axarr[0,0].axis('off')
    
    # mask frame after gaussian blur
    axarr[0,1].set_title('Mask frame', fontsize=20)
    axarr[0,1].imshow(mask_fr)
    axarr[0,1].axis('off')

    # background frame
    axarr[1,0].set_title('Background frame', fontsize=20)
    axarr[1,0].imshow(back_fr)
    axarr[1,0].axis('off')

    # ROI from the background frame
    axarr[1,1].set_title('ROI from the background frame', fontsize=20)
    axarr[1,1].imshow(back_pil)
    axarr[1,1].axis('off')

    # inverse mask
    axarr[2,0].set_title('Inverse of the mask', fontsize=20)
    axarr[2,0].imshow(mask_inv)
    axarr[2,0].axis('off')

    # threshold on inverse mask
    axarr[2,1].set_title('Thresholded inverse mask', fontsize=20)
    axarr[2,1].imshow(mask_thresh) 
    axarr[2,1].axis('off')

    # Morphological Close on mask
    axarr[3,0].set_title('Morphological close on mask', fontsize=20)
    axarr[3,0].imshow(mask_C)
    axarr[3,0].axis('off')

    # Dilate on mask
    axarr[3,1].set_title('Dilate on mask', fontsize=20)
    axarr[3,1].imshow(mask_D)
    axarr[3,1].axis('off')

    # Convert white pixels to transparent
    axarr[4,0].set_title('White pixels to transparent', fontsize=20)
    axarr[4,0].imshow(mask_pil)
    axarr[4,0].axis('off')

    # PIL image composite function
    axarr[4,1].set_title('PIL Image Composite', fontsize=20)
    axarr[4,1].imshow(dst_pil)
    axarr[4,1].axis('off')

    # display the full composed frame 
    axarr[5,0].set_title('Full frame display', fontsize=20)
    axarr[5,0].imshow(back_final)
    axarr[5,0].axis('off')      

  return back_final


#### Function 2 - compose_video_v2

In [ ]:
def compose_video_v2(fore_fr, mask_fr, back_fr, pos_x, pos_y, box):

  # mask shape
  mask_h = mask_fr.shape[0]
  mask_w = mask_fr.shape[1]

  # Check for the sizes to avoid errors
  if(pos_x + mask_w)>back_fr.shape[1] and (pos_y + mask_h)>back_fr.shape[0]:
      raise ValueError("Error in positioning. Try repositioning")
  
  # Get the ROI of the background for the mask positioning
  back_cr1 = back_fr[pos_y:pos_y+mask_h, pos_x:pos_x+mask_w, :]

  kernal = np.ones((3,3), np.uint8)
  mask_fr = cv2.erode(mask_fr, kernal, iterations=3)
  mask_fr2 = mask_fr.astype(np.float32)/255.0  
  back_final = (fore_fr*mask_fr2)+(back_cr1*(1-mask_fr2))

  return back_final

### Video Composition

#### Loading video files

In [ ]:
fore_f = 'path to foreground video'
mask_f = 'path to mask of foreground video'
back_f = 'path to desired background video'

fore = VideoFileClip(fore_f)
mask = VideoFileClip(mask_f)
back = VideoFileClip(back_f)

print("Loaded the video files")

Loaded the video files


#### Variables from the Video files

In [ ]:
# Video sizes
fore_w, fore_h = fore.size
mask_w, mask_h = mask.size
back_w, back_h = back.size

# Making the fps rate in foreground and mask as same as the bg
if (fore.fps!=back.fps or mask.fps!=back.fps):
  print("Frame rates mismatching; setting the frame rate to the background")
  fore = fore.set_fps(back.fps)
  mask = mask.set_fps(back.fps)

# if (fore_w!=back_w and fore_h!=back_h) or (mask_w!=back_w and mask_h!=back_h):
#   print("Foreground(&mask) video height and width are larger than the background video")
#   fore = fore.resize(height=back_h)
#   mask = mask.resize(height=back_h)
#   print("New height and width: ", fore.size)

#Get the fps of the video files
fore_fps = fore.fps
mask_fps = mask.fps
back_fps = back.fps

# Get the video durations
fore_t = fore.duration
mask_t = mask.duration
back_t = back.duration

# Calculate the number of frames of the videos
fore_nf = int(fore_fps * fore_t)
mask_nf = int(mask_fps * mask_t)
back_nf = int(back_fps * back_t)

# get the start and end frames of the bg video using the start time input
start_t = 0
start_bg_frame = int(back_fps * start_t)
end_bg_frame = int(start_bg_frame + mask_nf)
end_t = int(start_t + mask_t)

# Clip the background video for the duration
# Clip the foreground video to set for the mask video duration
back_vid = back.subclip(start_t, end_t)
fore_vid = fore.subclip(0, mask_t)

Frame rates mismatching; setting the frame rate to the background


#### Calling the function for each frame

In [ ]:
# Initialization of variables
i = 0
composed_clip = []
disp = False
disp_frame = 0
kernel = np.ones((3,3), np.uint8)

# positions
pos_x = 100
pos_y = 0

for fore_fr, mask_fr, back_fr in zip(fore_vid.iter_frames(), mask.iter_frames(), back_vid.iter_frames()):
    
  disp = False

  new_fr = compose_video_v2(fore_fr, mask_fr, back_fr, pos_x, pos_y, disp)
  back_fr[pos_y:pos_y+mask_h, pos_x:pos_x+mask_w, :] = new_fr

  composed_clip.append(back_fr)
  i += 1
  


#### Convert the frames array to video and display

In [ ]:
composed_video = 0
composed_video = ImageSequenceClip(composed_clip, fps=24)
composed_video.ipython_display(width=780)

#### Save the composed video

In [ ]:
fore_file_name = os.path.basename(fore_f)
fore_file_name = os.path.splitext(fore_file_name)[0]

composed_video_name = os.path.join('./composed_video',fore_file_name + '_composed.mp4')
composed_video.write_videofile(composed_video_name)